### Initial Setup


In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

### Tracing a Workflow Span


In [28]:
import requests
from ddtrace.llmobs.decorators import tool


# API docs: https://metmuseum.github.io/#search
@tool(name="fetch_met_urls")
def fetch_met_urls(query):
    base_url = "https://collectionapi.metmuseum.org/public/collection/v1/search"
    params = {
        "q": query,
        "hasImages": "true",
    }
    response = requests.get(base_url, params=params)
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        objects = response.json()["objectIDs"]
        return [
            f"https://www.metmuseum.org/art/collection/search/{objectId}"
            for objectId in objects[:5]
        ]
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Test it out here:
# response = call_api("cat")
# response

In [29]:
from openai import OpenAI
import os
import json
from ddtrace.llmobs.decorators import workflow


oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

sys_prompt = """
	You are a service that provides information about the art in the collection of the Metropolitan Museum of Art.
    You take a free text query and parse it into a format that can be sent to the Met API.
    Parse a query out from the user's message in the following JSON format:

    Prompt: I think I'd like to see some art from the 19th century. Ideally southern European or maybe Northern African.
    Response: {"query": "19th century southern European Northern African"}
	"""


def parse_query(message, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": message},
    ]
    response = (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )
    query = json.loads(response)["query"]
    print(f"Parsed query: {query}")
    return query


@workflow(name="get_art")
def get_art(question, prompt=sys_prompt):
    query = parse_query(question, prompt)
    urls = fetch_met_urls(query)
    return urls

In [30]:
get_art("I think I'd like to see some art from China that features a cat or doc")

Error submitting packet: [Errno 61] Connection refused, dropping the packet and closing the socket, 8 additional messages skipped


Parsed query: China cat dog


['https://www.metmuseum.org/art/collection/search/49698',
 'https://www.metmuseum.org/art/collection/search/36221',
 'https://www.metmuseum.org/art/collection/search/49470',
 'https://www.metmuseum.org/art/collection/search/482595',
 'https://www.metmuseum.org/art/collection/search/61658']

failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 2 additional messages skipped
